In [43]:
import numpy as np

def axisAngle2quatern(axis, angle):
    q0 = np.cos(angle/2)
    q1 = -axis[:,0]*np.sin(angle/2)
    q2 = -axis[:,1]*np.sin(angle/2)
    q3 = -axis[:,2]*np.sin(angle/2)
    q = np.column_stack((q0, q1, q2, q3))
    return q

def axisAngle2rotMat(axis, angle):
    kx = axis[:,0]
    ky = axis[:,1]
    kz = axis[:,2]
    cT = np.cos(angle)
    sT = np.sin(angle)
    vT = 1 - np.cos(angle)
    R = np.zeros((3, 3, axis.shape[0]))

    R[0,0,:] = kx*kx*vT + cT
    R[0,1,:] = kx*ky*vT - kz*sT
    R[0,2,:] = kx*kz*vT + ky*sT

    R[1,0,:] = kx*ky*vT + kz*sT
    R[1,1,:] = ky*ky*vT + cT
    R[1,2,:] = ky*kz*vT - kx*sT

    R[2,0,:] = kx*kz*vT - ky*sT
    R[2,1,:] = ky*kz*vT + kx*sT
    R[2,2,:] = kz*kz*vT + cT
    return R


def euler2rotMat(phi, theta, psi):
    R = np.zeros((3, 3, len(phi)))
    R[0, 0, :] = np.cos(psi) * np.cos(theta)
    R[0, 1, :] = -np.sin(psi) * np.cos(phi) + np.cos(psi) * np.sin(theta) * np.sin(phi)
    R[0, 2, :] = np.sin(psi) * np.sin(phi) + np.cos(psi) * np.sin(theta) * np.cos(phi)

    R[1, 0, :] = np.sin(psi) * np.cos(theta)
    R[1, 1, :] = np.cos(psi) * np.cos(phi) + np.sin(psi) * np.sin(theta) * np.sin(phi)
    R[1, 2, :] = -np.cos(psi) * np.sin(phi) + np.sin(psi) * np.sin(theta) * np.cos(phi)

    R[2, 0, :] = -np.sin(theta)
    R[2, 1, :] = np.cos(theta) * np.sin(phi)
    R[2, 2, :] = np.cos(theta) * np.cos(phi)
    return R

def quatern2euler(q):
    R = np.zeros((3, 3, q.shape[0]))
    R[0, 0, :] = 2 * q[:, 0] ** 2 - 1 + 2 * q[:, 1] ** 2
    R[1, 0, :] = 2 * (q[:, 1] * q[:, 2] - q[:, 0] * q[:, 3])
    R[2, 0, :] = 2 * (q[:, 1] * q[:, 3] + q[:, 0] * q[:, 2])
    R[2, 1, :] = 2 * (q[:, 2] * q[:, 3] - q[:, 0] * q[:, 1])
    R[2, 2, :] = 2 * q[:, 0] ** 2 - 1 + 2 * q[:, 3] ** 2

    phi = np.arctan2(R[2, 1, :], R[2, 2, :])
    theta = -np.arctan(R[2, 0, :] / np.sqrt(1 - R[2, 0, :] ** 2))
    psi = np.arctan2(R[1, 0, :], R[0, 0, :])
    
    euler = np.column_stack((phi[0, :], theta[0, :], psi[0, :]))
    return euler



def quatern2rotMat(q):
    rows, cols = q.shape
    R = np.zeros((3, 3, rows))

    R[0, 0, :] = 2 * q[:, 0]**2 - 1 + 2 * q[:, 1]**2
    R[0, 1, :] = 2 * (q[:, 1] * q[:, 2] + q[:, 0] * q[:, 3])
    R[0, 2, :] = 2 * (q[:, 1] * q[:, 3] - q[:, 0] * q[:, 2])

    R[1, 0, :] = 2 * (q[:, 1] * q[:, 2] - q[:, 0] * q[:, 3])
    R[1, 1, :] = 2 * q[:, 0]**2 - 1 + 2 * q[:, 2]**2
    R[1, 2, :] = 2 * (q[:, 2] * q[:, 3] + q[:, 0] * q[:, 1])

    R[2, 0, :] = 2 * (q[:, 1] * q[:, 3] + q[:, 0] * q[:, 2])
    R[2, 1, :] = 2 * (q[:, 2] * q[:, 3] - q[:, 0] * q[:, 1])
    R[2, 2, :] = 2 * q[:, 0]**2 - 1 + 2 * q[:, 3]**2

    return R

def quaternConj(q):
    qConj = np.array([q[0], -q[1], -q[2], -q[3]])
    return qConj

def quaternProd(a, b):
    ab = np.zeros((a.shape[0], 4))
    ab[:, 0] = a[:, 0] * b[:, 0] - a[:, 1] * b[:, 1] - a[:, 2] * b[:, 2] - a[:, 3] * b[:, 3]
    ab[:, 1] = a[:, 0] * b[:, 1] + a[:, 1] * b[:, 0] + a[:, 2] * b[:, 3] - a[:, 3] * b[:, 2]
    ab[:, 2] = a[:, 0] * b[:, 2] - a[:, 1] * b[:, 3] + a[:, 2] * b[:, 0] + a[:, 3] * b[:, 1]
    ab[:, 3] = a[:, 0] * b[:, 3] + a[:, 1] * b[:, 2] - a[:, 2] * b[:, 1] + a[:, 3] * b[:, 0]
    return ab

def quaternRotate(v, q):
    row, col = v.shape
    v0XYZ = quaternProd(quaternProd(q, np.hstack([np.zeros((row, 1)), v.T])).T, quaternConj(q))
    v = v0XYZ[:, 1:4]
    return v

def rotMat2euler(R):
    phi = np.arctan2(R[2, 1, :], R[2, 2, :])
    theta = -np.arctan(R[2, 0, :] / np.sqrt(1 - R[2, 0, :]**2))
    psi = np.arctan2(R[1, 0, :], R[0, 0, :])

    euler = np.column_stack((phi, theta, psi))
    return euler

def rotMat2quatern(R):
    numR = R.shape[2]
    q = np.zeros((numR, 4))
    for i in range(numR):
        K = np.zeros((4, 4))
        K[0, 0] = (1/3) * (R[0, 0, i] - R[1, 1, i] - R[2, 2, i])
        K[0, 1] = (1/3) * (R[1, 0, i] + R[0, 1, i])
        K[0, 2] = (1/3) * (R[2, 0, i] + R[0, 2, i])
        K[0, 3] = (1/3) * (R[1, 2, i] - R[2, 1, i])
        K[1, 0] = (1/3) * (R[1, 0, i] + R[0, 1, i])
        K[1, 1] = (1/3) * (R[1, 1, i] - R[0, 0, i] - R[2, 2, i])
        K[1, 2] = (1/3) * (R[2, 1, i] + R[1, 2, i])
        K[1, 3] = (1/3) * (R[2, 0, i] - R[0, 2, i])
        K[2, 0] = (1/3) * (R[2, 0, i] + R[0, 2, i])
        K[2, 1] = (1/3) * (R[2, 1, i] + R[1, 2, i])
        K[2, 2] = (1/3) * (R[2, 2, i] - R[0, 0, i] - R[1, 1, i])
        K[2, 3] = (1/3) * (R[0, 1, i] - R[1, 0, i])
        K[3, 0] = (1/3) * (R[1, 2, i] - R[2, 1, i])
        K[3, 1] = (1/3) * (R[2, 0, i] - R[0, 2, i])
        K[3, 2] = (1/3) * (R[0, 1, i] - R[1, 0, i])
        K[3, 3] = (1/3) * (R[0, 0, i] + R[1, 1, i] + R[2, 2, i])
        
        eigvals, eigvecs = np.linalg.eig(K)
        p = np.argmax(eigvals)
        q[i, :] = eigvecs[:, p]
        q[i, :] = [q[i, 3], q[i, 0], q[i, 1], q[i, 2]]
    
    return q


In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import csv
import os
import glob
import re
from collections import OrderedDict
from abc import ABC, abstractmethod

class DataBaseClass:
    def __init__(self):
        self.FileNameAppendage = None
        self.NumPackets = 0
        self.PacketNumber = []

    def ImportCSVnumeric(self, fileNamePrefix):
        data = np.loadtxt(self.CreateFileName(fileNamePrefix), delimiter=',', skiprows=1)
        self.PacketNumber = data[:, 0]
        self.NumPackets = len(self.PacketNumber)
        return data

    def ImportCSVmixed(self, fileNamePrefix, fieldSpecifier):
        with open(self.CreateFileName(fileNamePrefix)) as f:
            lines = f.readlines()[1:]  # Skip column headings
            data = [line.strip().split(',') for line in lines]
            self.PacketNumber = [float(row[0]) for row in data]
            self.NumPackets = len(self.PacketNumber)
            formatted_data = [tuple(row) for row in data]
            return np.array(formatted_data, dtype=fieldSpecifier)

    def CreateFigName(self):
        pathstr, name, ext, versn = os.path.splitext(self.FileNameAppendage)
        figName = name[1:]
        return figName

    def CreateFileName(self, fileNamePrefix):
        fileName = f"{fileNamePrefix}{self.FileNameAppendage}"
        if not os.path.isfile(fileName):
            raise FileNotFoundError(f"File not found: {fileName}. No data was imported.")
        return fileName

class TimeSeriesDataBaseClass(DataBaseClass, ABC):
    def __init__(self):
        super().__init__()
        self._Time = []
        self._SamplePeriod = 0
        self._SampleRate = 0
        self._StartTime = 0
        self._TimeAxis = None

    @property
    def FileNameAppendage(self):
        raise NotImplementedError("Subclasses should implement this!")

    @property
    def Time(self):
        return self._Time

    @property
    def SamplePeriod(self):
        if self._SampleRate == 0:
            return 0
        else:
            return 1 / self._SampleRate

    @property
    def SampleRate(self):
        return self._SampleRate

    @SampleRate.setter
    def SampleRate(self, value):
        self._SampleRate = value
        if self._SampleRate == 0:
            self._Time = []
            self._TimeAxis = 'Sample'
        elif self.NumPackets != 0:
            self._Time = [i * (1/self._SampleRate) + self._StartTime for i in range(self.NumPackets)]
            self._TimeAxis = 'Time (s)'

    @property
    def StartTime(self):
        return self._StartTime

    @StartTime.setter
    def StartTime(self, value):
        self._StartTime = value
        self.SampleRate = self._SampleRate  # This will update the Time and TimeAxis accordingly

    @property
    def TimeAxis(self):
        return self._TimeAxis

    @abstractmethod
    def Plot(self):
        pass

class InertialAndMagneticDataBaseClass(TimeSeriesDataBaseClass, ABC):
    def __init__(self):
        super().__init__()
        self.Gyroscope = {'X': [], 'Y': [], 'Z': []}
        self.Accelerometer = {'X': [], 'Y': [], 'Z': []}
        self.Magnetometer = {'X': [], 'Y': [], 'Z': []}
        self._GyroscopeUnits = None
        self._AccelerometerUnits = None
        self._MagnetometerUnits = None

    @property
    def FileNameAppendage(self):
        raise NotImplementedError("Subclasses should implement this!")

    @property
    def GyroscopeUnits(self):
        return self._GyroscopeUnits

    @GyroscopeUnits.setter
    def GyroscopeUnits(self, value):
        self._GyroscopeUnits = value

    @property
    def AccelerometerUnits(self):
        return self._AccelerometerUnits

    @AccelerometerUnits.setter
    def AccelerometerUnits(self, value):
        self._AccelerometerUnits = value

    @property
    def MagnetometerUnits(self):
        return self._MagnetometerUnits

    @MagnetometerUnits.setter
    def MagnetometerUnits(self, value):
        self._MagnetometerUnits = value

    def Import(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix)
        self.Gyroscope['X'] = data[:, 1]
        self.Gyroscope['Y'] = data[:, 2]
        self.Gyroscope['Z'] = data[:, 3]
        self.Accelerometer['X'] = data[:, 4]
        self.Accelerometer['Y'] = data[:, 5]
        self.Accelerometer['Z'] = data[:, 6]
        self.Magnetometer['X'] = data[:, 7]
        self.Magnetometer['Y'] = data[:, 8]
        self.Magnetometer['Z'] = data[:, 9]
        self.SampleRate = self.SampleRate  # This will update the Time and TimeAxis accordingly

    def Plot(self):
        import matplotlib.pyplot as plt
        if self.NumPackets == 0:
            raise ValueError('No data to plot.')
        else:
            time = list(range(1, self.NumPackets + 1)) if not self.Time else self.Time
            fig, ax = plt.subplots(3, 1, figsize=(8, 10))
            colors = ['r', 'g', 'b']
            # Gyroscope plot
            for i, axis in enumerate(['X', 'Y', 'Z']):
                ax[0].plot(time, self.Gyroscope[axis], colors[i], label=axis)
            ax[0].set_xlabel(self.TimeAxis)
            ax[0].set_ylabel(f'Angular rate ({self.GyroscopeUnits})')
            ax[0].set_title('Gyroscope')
            ax[0].legend()

            # Accelerometer plot
            for i, axis in enumerate(['X', 'Y', 'Z']):
                ax[1].plot(time, self.Accelerometer[axis], colors[i], label=axis)
            ax[1].set_xlabel(self.TimeAxis)
            ax[1].set_ylabel(f'Acceleration ({self.AccelerometerUnits})')
            ax[1].set_title('Accelerometer')
            ax[1].legend()

            # Magnetometer plot
            for i, axis in enumerate(['X', 'Y', 'Z']):
                ax[2].plot(time, self.Magnetometer[axis], colors[i], label=axis)
            ax[2].set_xlabel(self.TimeAxis)
            ax[2].set_ylabel(f'Flux ({self.MagnetometerUnits})')
            ax[2].set_title('Magnetometer')
            ax[2].legend()

            plt.tight_layout()
            return fig

class ADXL345busDataBaseClass(TimeSeriesDataBaseClass):
    def __init__(self):
        super().__init__()
        self.AccelerometerUnits = ""  # Fill in with the appropriate units
    
    def Import(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix)
        self.ADXL345A = {
            'X': data[:, 1],
            'Y': data[:, 2],
            'Z': data[:, 3]
        }
        self.ADXL345B = {
            'X': data[:, 4],
            'Y': data[:, 5],
            'Z': data[:, 6]
        }
        self.ADXL345C = {
            'X': data[:, 7],
            'Y': data[:, 8],
            'Z': data[:, 9]
        }
        self.ADXL345D = {
            'X': data[:, 10],
            'Y': data[:, 11],
            'Z': data[:, 12]
        }
        self.SampleRate = self.SampleRate  # Call the set method to create the time vector

    def Plot(self):
        if self.NumPackets == 0:
            raise ValueError('No data to plot.')
        else:
            if self.Time is None:
                time = np.arange(1, self.NumPackets + 1)
            else:
                time = self.Time
            fig, ax = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
            
            def plot_acceleration(ax, data, label):
                ax.plot(time, data['X'], 'r', label='X')
                ax.plot(time, data['Y'], 'g', label='Y')
                ax.plot(time, data['Z'], 'b', label='Z')
                ax.legend()
                ax.set_xlabel(self.TimeAxis)
                ax.set_ylabel(f'Acceleration ({self.AccelerometerUnits})')
                ax.set_title(label)
            
            plot_acceleration(ax[0], self.ADXL345A, 'ADXL345 A')
            plot_acceleration(ax[1], self.ADXL345B, 'ADXL345 B')
            plot_acceleration(ax[2], self.ADXL345C, 'ADXL345 C')
            plot_acceleration(ax[3], self.ADXL345D, 'ADXL345 D')
            
            plt.tight_layout()
            plt.show()

class AnalogueInputDataBaseClass(TimeSeriesDataBaseClass):
    def __init__(self):
        super().__init__()
        self.AX0 = []
        self.AX1 = []
        self.AX2 = []
        self.AX3 = []
        self.AX4 = []
        self.AX5 = []
        self.AX6 = []
        self.AX7 = []

    def Import(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix)
        self.AX0 = data[:, 1]
        self.AX1 = data[:, 2]
        self.AX2 = data[:, 3]
        self.AX3 = data[:, 4]
        self.AX4 = data[:, 5]
        self.AX5 = data[:, 6]
        self.AX6 = data[:, 7]
        self.AX7 = data[:, 8]
        self.SampleRate = self.SampleRate  # call set method to create time vector

    def Plot(self):
        if self.NumPackets == 0:
            raise ValueError('No data to plot.')
        else:
            time = list(range(1, self.NumPackets + 1)) if not self.Time else self.Time
            fig = plt.figure()
            plt.plot(time, self.AX0, 'r')
            plt.plot(time, self.AX1, 'g')
            plt.plot(time, self.AX2, 'b')
            plt.plot(time, self.AX3, 'k')
            plt.plot(time, self.AX4, ':r')
            plt.plot(time, self.AX5, ':g')
            plt.plot(time, self.AX6, ':b')
            plt.plot(time, self.AX7, ':k')
            plt.xlabel(self.TimeAxis)
            plt.ylabel(f'Voltage ({self.ADCunits})')
            plt.title('Analogue Input')
            return fig

class BatteryAndThermometerDataBaseClass(TimeSeriesDataBaseClass):
    def __init__(self):
        super().__init__()
        self.Battery = []
        self.Thermometer = []
    
    '''
    def Import(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix)
        self.Battery = data[:, 1]
        self.Thermometer = data[:, 2]
        self.SampleRate = self.SampleRate  # call set method to create time vector
    '''
    def Import(self, filename):
        with open(filename, "r") as f:
            for line in f:
                data = line.split()
                if data[0] == "battery_voltage":
                    self._battery_voltage = float(data[1])
                elif data[0] == "temperature":
                    self._temperature = float(data[1])
        self._data = data

    def Plot(self):
        if self.NumPackets == 0:
            raise ValueError('No data to plot.')
        else:
            time = list(range(1, self.NumPackets + 1)) if not self.Time else self.Time
            fig, ax = plt.subplots(2, 1, figsize=(8, 6))
            ax[0].plot(time, self.Battery)
            ax[0].set_xlabel(self.TimeAxis)
            ax[0].set_ylabel(f'Voltage ({self.BatteryUnits})')
            ax[0].set_title('Battery Voltmeter')
            ax[1].plot(time, self.Thermometer)
            ax[1].set_xlabel(self.TimeAxis)
            ax[1].set_ylabel(f'Temperature ({self.ThermometerUnits})')
            ax[1].set_title('Thermometer')
            plt.tight_layout()
            return fig

class CalADXL345busDataClass(ADXL345busDataBaseClass):
    def __init__(self, fileNamePrefix, SampleRate=None, *varargin):
        super().__init__()
        self.FileNameAppendage = '_CalADXL345bus.csv'
        if SampleRate is not None:
            self.SampleRate = SampleRate
        for i in range(2, len(varargin), 2):
            if varargin[i] == 'SampleRate':
                self.SampleRate = varargin[i + 1]
            else:
                raise ValueError('Invalid argument.')
        self.Import(fileNamePrefix)
        self.AccelerometerUnits = 'g'

class CalAnalogueInputDataClass(AnalogueInputDataBaseClass):
    def __init__(self, fileNamePrefix, SampleRate=None, *varargin):
        super().__init__()
        self.FileNameAppendage = '_CalAnalogueInput.csv'
        if SampleRate is not None:
            self.SampleRate = SampleRate
        for i in range(2, len(varargin), 2):
            if varargin[i] == 'SampleRate':
                self.SampleRate = varargin[i + 1]
            else:
                raise ValueError('Invalid argument.')
        self.Import(fileNamePrefix)
        self.ADCunits = 'V'

class CalBatteryAndThermometerDataClass(BatteryAndThermometerDataBaseClass):
    def __init__(self, fileNamePrefix, SampleRate=None, *varargin):
        super().__init__()
        self.FileNameAppendage = '_CalBattAndTherm.csv'
        if SampleRate is not None:
            self.SampleRate = SampleRate
        for i in range(2, len(varargin), 2):
            if varargin[i] == 'SampleRate':
                self.SampleRate = varargin[i + 1]
            else:
                raise ValueError('Invalid argument.')
        self.Import(fileNamePrefix)
        self.ThermometerUnits = '^\circC'
        self.BatteryUnits = 'G'

'''
class CalInertialAndMagneticDataClass(InertialAndMagneticDataBaseClass):
    def __init__(self, fileNamePrefix, SampleRate=None, *varargin):
        super().__init__()
        self.FileNameAppendage = '_CalInertialAndMag.csv'
        if SampleRate is not None:
            self.SampleRate = SampleRate
        for i in range(2, len(varargin), 2):
            if varargin[i] == 'SampleRate':
                self.SampleRate = varargin[i + 1]
            else:
                raise ValueError('Invalid argument.')
        self.Import(fileNamePrefix)
        self.GyroscopeUnits = '^\circ/s'
        self.AccelerometerUnits = 'g'
        self.MagnetometerUnits = 'G'
        self._SampleRate = None  # _SampleRate 속성을 명시적으로 초기화

    @property
    def SampleRate(self):
        return self._SampleRate

    @SampleRate.setter
    def SampleRate(self, value):
        self._SampleRate = value
'''

class CalInertialAndMagneticDataClass(InertialAndMagneticDataBaseClass):

    def __init__(self, filename, filetype):
        super().__init__(filename, filetype)

    @property
    def gyroscope(self):
        return self._gyroscope

    @gyroscope.setter
    def gyroscope(self, new_gyroscope):
        self._gyroscope = new_gyroscope

    @property
    def magnetometer(self):
        return self._magnetometer

    @magnetometer.setter
    def magnetometer(self, new_magnetometer):
        self._magnetometer = new_magnetometer

class CommandDataClass(DataBaseClass):
    def __init__(self, fileNamePrefix):
        super().__init__()
        self.FileNameAppendage = '_Commands.csv'
        self.Code = []
        self.Message = []
        data = self.ImportCSVmixed(fileNamePrefix, '%f %f %s')
        self.Code = data[1]
        self.Message = data[2]

class DateTimeDataClass(TimeSeriesDataBaseClass):
    def __init__(self, fileNamePrefix, SampleRate=None):
        self.FileNameAppendage = '_DateTime.csv'
        self.String = []
        self.Vector = []
        self.Serial = []

        if SampleRate is not None:
            self.SampleRate = SampleRate

        data = self.ImportCSVnumeric(fileNamePrefix)
        self.Vector = data[:, 1:7]
        self.String = [datetime(*vec).strftime("%Y-%m-%d %H:%M:%S") for vec in self.Vector]
        self.Serial = [datetime(*vec) for vec in self.Vector]

        if SampleRate is not None:
            self.SampleRate = self.SampleRate  # Call set method to create time vector

    def Plot(self):
        raise NotImplementedError("This method is unimplemented.")

class DigitalIODataClass(TimeSeriesDataBaseClass):
    def __init__(self, *args):
        super().__init__(*args)
        self.FileNameAppendage = '_DigitalIO.csv'
        self.Direction = {'AX0': [], 'AX1': [], 'AX2': [], 'AX3': [], 'AX4': [], 'AX5': [], 'AX6': [], 'AX7': []}
        self.State = {'AX0': [], 'AX1': [], 'AX2': [], 'AX3': [], 'AX4': [], 'AX5': [], 'AX6': [], 'AX7': []}

        fileNamePrefix = args[0]
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i+1]
            else:
                raise ValueError('Invalid argument.')

        data = self.ImportCSVnumeric(fileNamePrefix)
        self.Direction['AX0'] = data[:, 2]
        self.Direction['AX1'] = data[:, 3]
        self.Direction['AX2'] = data[:, 4]
        self.Direction['AX3'] = data[:, 5]
        self.Direction['AX4'] = data[:, 6]
        self.Direction['AX5'] = data[:, 7]
        self.Direction['AX6'] = data[:, 8]
        self.Direction['AX7'] = data[:, 9]
        self.State['AX0'] = data[:, 10]
        self.State['AX1'] = data[:, 11]
        self.State['AX2'] = data[:, 12]
        self.State['AX3'] = data[:, 13]
        self.State['AX4'] = data[:, 14]
        self.State['AX5'] = data[:, 15]
        self.State['AX6'] = data[:, 16]
        self.State['AX7'] = data[:, 17]
        self.SampleRate = self.SampleRate  # call set method to create time vector

    def Plot(self):
        if self.NumPackets == 0:
            raise ValueError('No data to plot.')
        else:
            if self.Time is None:
                time = range(1, self.NumPackets + 1)
            else:
                time = self.Time
            fig = plt.figure('Name', self.CreateFigName())
            plt.hold(True)
            plt.plot(time, self.State['AX0'], 'r')
            plt.plot(time, self.State['AX1'], 'g')
            plt.plot(time, self.State['AX2'], 'b')
            plt.plot(time, self.State['AX3'], 'k')
            plt.plot(time, self.State['AX4'], ':r')
            plt.plot(time, self.State['AX5'], ':g')
            plt.plot(time, self.State['AX6'], ':b')
            plt.plot(time, self.State['AX7'], ':k')
            plt.title('Digital I/O')
            plt.xlabel(self.TimeAxis)
            plt.ylabel('State (Binary)')
            plt.legend(['AX0', 'AX1', 'AX2', 'AX3', 'AX4', 'AX5', 'AX6', 'AX7'])
            plt.hold(False)
        return fig




class ErrorDataClass(DataBaseClass):
    # Public 'read-only' properties
    FileNameAppendage = '_Errors.csv'
    Code = []
    Message = []

    # Public methods
    def __init__(self, fileNamePrefix):
        super().__init__()  # Call the constructor of the base class
        data = self.ImportCSVmixed(fileNamePrefix, '%f %f %s')
        self.Code = data[1]  # MATLAB uses 1-based indexing
        self.Message = data[2]  # MATLAB uses 1-based indexing

class EulerAnglesDataClass:
    _fileNameAppendage = '_EulerAngles.csv'

    def __init__(self, fileNamePrefix, **kwargs):
        self.sampleRate = kwargs.get('SampleRate', None)
        self.phi = []
        self.theta = []
        self.psi = []
        self.time = None
        self.numPackets = 0

        data = self._import_csv_numeric(fileNamePrefix)
        self.phi = data[:, 1]
        self.theta = data[:, 2]
        self.psi = data[:, 3]
        if self.sampleRate is not None:
            self.time = np.arange(0, len(data) / self.sampleRate, 1 / self.sampleRate)
            self.numPackets = len(self.time)

    def _import_csv_numeric(self, fileNamePrefix):
        with open(f'{fileNamePrefix}{self._fileNameAppendage}', 'r') as file:
            reader = csv.reader(file)
            next(reader, None)  # skip the header
            return np.array(list(reader)).astype(np.float)

    def plot(self):
        if self.numPackets == 0:
            raise ValueError('No data to plot.')
        else:
            time = self.time if self.time is not None else np.arange(1, self.numPackets + 1)
            plt.figure('Euler Angles')
            plt.plot(time, self.phi, 'r', label='$\\phi$')
            plt.plot(time, self.theta, 'g', label='$\\theta$')
            plt.plot(time, self.psi, 'b', label='$\\psi$')
            plt.title('Euler angles')
            plt.xlabel('Time' if self.time is not None else 'Packet Number')
            plt.ylabel('Angle (degrees)')
            plt.legend()
            plt.show()

class ImportDirectory:
    def __init__(self, directory):
        self.directory = directory

    def _list_csv_files(self):
        return glob.glob(os.path.join(self.directory, '*_*.csv'))

    def _load_data_from_csv(self, file_path):
        data = {}
        with open(file_path, 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader, None)  # Skip the header row if there is one
            for row in csv_reader:
                if len(row) >= 2:  # Ensure the row has at least two elements
                    address = int(row[0])  # Assuming the first column is the address
                    value = row[1]  # Assuming the second column is the value
                    data[address] = value  # Store the data
        return data

    def _import_data_objects(self, file_paths):
        ximu_data_objs = {}
        for file_path in file_paths:
            prefix = re.match(r'([^_]+)_', os.path.basename(file_path))
            if prefix:
                try:
                    data = self._load_data_from_csv(file_path)
                    ximu_data_objs[prefix.group(1)] = data
                except Exception as e:
                    print(f"Failed to load {file_path}: {e}")
        return ximu_data_objs

    def _generate_field_names(self, ximu_data_objs):
        ximu_data_struct = OrderedDict()
        for prefix, data in ximu_data_objs.items():
            try:
                device_id = format(data.get(2), 'X')  # Assuming address 2 contains the device ID
                field_name = f'ID_{device_id}'
            except Exception:
                alphanumeric_prefix = ''.join(filter(str.isalnum, prefix))
                field_name = f'FILE_{alphanumeric_prefix}'
            ximu_data_struct[field_name] = data
        return ximu_data_struct

    def import_directory(self):
        file_paths = self._list_csv_files()
        if not file_paths:
            raise ValueError('No CSV files found in the directory.')
        ximu_data_objs = self._import_data_objects(file_paths)
        if not ximu_data_objs:
            raise ValueError('No data was imported.')
        return self._generate_field_names(ximu_data_objs)
        
class PWMoutputDataClass(DataBaseClass):
    def __init__(self, fileNamePrefix):
        super().__init__()
        self._FileNameAppendage = '_PWMoutput.csv'
        self._AX0 = []
        self._AX2 = []
        self._AX4 = []
        self._AX6 = []
        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    @property
    def AX0(self):
        return self._AX0

    @property
    def AX2(self):
        return self._AX2

    @property
    def AX4(self):
        return self._AX4

    @property
    def AX6(self):
        return self._AX6

    def import_data(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix + self.FileNameAppendage)
        self._AX0 = data[:, 1]
        self._AX2 = data[:, 2]
        self._AX4 = data[:, 3]
        self._AX6 = data[:, 4]

class QuaternionDataClass(TimeSeriesDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_Quaternion.csv'
        self._Quaternion = []

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)
        self.SampleRate = self.SampleRate  # 이 부분은 SampleRate 설정에 따른 추가 로직이 필요할 수 있습니다.

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    @property
    def Quaternion(self):
        return self._Quaternion

    def import_data(self, fileNamePrefix):
        data = self.ImportCSVnumeric(fileNamePrefix + self.FileNameAppendage)
        self._Quaternion = data[:, 1:5]  # 쿼터니언 데이터를 추출합니다.

    def Plot(self):
        raise NotImplementedError('This method is unimplemented.')
    
class RawADXL345busDataClass(ADXL345busDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_RawADXL345bus.csv'
        self.AccelerometerUnits = 'g'  # Protected parent class variable

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        # 이 메소드는 ADXL345busDataBaseClass에 정의된 Import 메소드를 호출합니다.
        self.Import(fileNamePrefix + self.FileNameAppendage)

class RawAnalogueInputDataClass(AnalogueInputDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_RawAnalogueInput.csv'
        self.ADCunits = 'lsb'  # Protected parent class variable

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        # 이 메소드는 AnalogueInputDataBaseClass에 정의된 Import 메소드를 호출합니다.
        self.Import(fileNamePrefix + self.FileNameAppendage)

class RawBatteryAndThermometerDataClass(BatteryAndThermometerDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_RawBattAndTherm.csv'
        self.ThermometerUnits = 'lsb'  # Protected parent class variable
        self.BatteryUnits = 'lsb'  # Protected parent class variable

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        # 이 메소드는 BatteryAndThermometerDataBaseClass에 정의된 Import 메소드를 호출합니다.
        self.Import(fileNamePrefix + self.FileNameAppendage)

class RawInertialAndMagneticDataClass(InertialAndMagneticDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_RawInertialAndMag.csv'
        self.GyroscopeUnits = 'lsb'  # Protected parent class variable
        self.AccelerometerUnits = 'lsb'  # Protected parent class variable
        self.MagnetometerUnits = 'lsb'  # Protected parent class variable

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        # 이 메소드는 InertialAndMagneticDataBaseClass에 정의된 Import 메소드를 호출합니다.
        self.Import(fileNamePrefix + self.FileNameAppendage)

class RegisterDataClass(DataBaseClass):
    def __init__(self, fileNamePrefix):
        super().__init__()
        self._FileNameAppendage = '_Registers.csv'
        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        # Pandas를 사용하여 CSV 파일에서 혼합 데이터 유형을 가져옵니다.
        df = pd.read_csv(fileNamePrefix + self.FileNameAppendage, header=None)
        self.Address = df[0].values
        self.Value = df[1].values
        self.FloatValue = df[2].values
        self.Name = df[3].values

    def GetValueAtAddress(self, address):
        return self.ValueAtIndexes(self.IndexesOfAddress(address))

    def GetFloatValueAtAddress(self, address):
        return self.FloatValueAtIndexes(self.IndexesOfAddress(address))

    def GetValueAtName(self, name):
        return self.ValueAtIndexes(self.IndexesOfName(name))

    def GetFloatValueAtName(self, name):
        return self.FloatValueAtIndexes(self.IndexesOfName(name))

    def IndexesOfAddress(self, address):
        indexes = np.where(self.Address == address)[0]
        if len(indexes) == 0:
            raise ValueError('Register address not found.')
        return indexes

    def IndexesOfName(self, name):
        indexes = np.where(self.Name == name)[0]
        if len(indexes) == 0:
            raise ValueError('Register name not found.')
        return indexes

    def ValueAtIndexes(self, indexes):
        unique_values = np.unique(self.Value[indexes])
        if len(unique_values) > 1:
            raise ValueError('Conflicting register values exist.')
        return unique_values[0]

    def FloatValueAtIndexes(self, indexes):
        unique_values = np.unique(self.FloatValue[indexes])
        if len(unique_values) > 1:
            raise ValueError('Conflicting register values exist.')
        return unique_values[0]

class RotationMatrixDataClass(TimeSeriesDataBaseClass):
    def __init__(self, *args):
        super().__init__()
        self._FileNameAppendage = '_RotationMatrix.csv'
        self.RotationMatrix = None

        fileNamePrefix = args[0]
        self.SampleRate = 0  # 기본 샘플률 설정
        for i in range(1, len(args), 2):
            if args[i] == 'SampleRate':
                self.SampleRate = args[i + 1]
            else:
                raise ValueError('Invalid argument.')

        self.import_data(fileNamePrefix)

    @property
    def FileNameAppendage(self):
        return self._FileNameAppendage

    def import_data(self, fileNamePrefix):
        data = pd.read_csv(fileNamePrefix + self.FileNameAppendage, header=None)
        num_packets = len(data)
        self.RotationMatrix = np.zeros((3, 3, num_packets))
        for i in range(1, 10):  # 데이터 열을 반복합니다.
            row, col = (i - 1) // 3, (i - 1) % 3
            self.RotationMatrix[row, col, :] = data[i].values

        self.SampleRate = self.SampleRate  # 이 부분은 SampleRate 설정에 따른 추가 로직이 필요할 수 있습니다.

    def Plot(self):
        raise NotImplementedError('This method is unimplemented.')


def SyncroniseData(*args):
    """
    Syncronises time series data between xIMUdataClass objects by adjusting the StartTime and SampleRate properties of each data class.
    """

    # Apply arguments
    xIMUdata = args[0]
    xIMUdataObjs = list(xIMUdata.values())
    StartEventTimes = []
    EndEventTimes = []
    UseAX0fallingEdge = False
    if isinstance(args[1], str):
        if args[1] == 'UseAX0fallingEdge':
            UseAX0fallingEdge = True
        else:
            raise ValueError('Invalid argument.')
    else:
        StartEventTimes = args[1]
        if len(args) == 3:
            EndEventTimes = args[2]

    # Use AX0 falling edge of auxiliary port in Digital I/O mode
    if UseAX0fallingEdge:
        for xIMUobj in xIMUdataObjs:
            fallingEdgeIndexes = np.where(np.diff(xIMUobj.DigitalIOdata.State.AX0) == -1)[0]
            fallingEdgeTimes = xIMUobj.DigitalIOdata.Time[fallingEdgeIndexes]
            StartEventTimes.append(fallingEdgeTimes[0])
            if len(fallingEdgeTimes) > 1:
                EndEventTimes.append(fallingEdgeTimes[-1])

    # Modify start times to synchronise start of window
    if len(StartEventTimes) != len(xIMUdataObjs):
        raise ValueError('Length of StartEventTimes vector must equal number of xIMUdataClass objects')
    for i, xIMUobj in enumerate(xIMUdataObjs):
        try:
            xIMUobj.DateTimeData.StartTime = -StartEventTimes[i]
            # ... Similar assignments for other data classes ...
        except AttributeError as e:
            pass  # Handle the exception if the attribute doesn't exist

    # Modify sample rate to synchronise end of window
    if len(EndEventTimes) == 0:
        return
    if len(EndEventTimes) != len(xIMUdataObjs):
        raise ValueError('Length of EndEventTimes vector must equal number of xIMUdataClass objects')
    scalers = (np.array(EndEventTimes) - np.array(StartEventTimes)) / (EndEventTimes[0] - StartEventTimes[0])
    for i, xIMUobj in enumerate(xIMUdataObjs[1:], start=1):
        try:
            new_sample_rate = scalers[i] * xIMUobj.SampleRate
            xIMUobj.SampleRate = new_sample_rate
            # ... Similar assignments for other data classes ...
        except AttributeError as e:
            pass  # Handle the exception if the attribute doesn't exist


'''
class xIMUdataClass:
    def __init__(self, file_name_prefix, *args):
        self.FileNamePrefix = file_name_prefix
        # 모든 데이터 클래스의 인스턴스를 None으로 초기화합니다.
        self.ErrorData = None
        self.CommandData = None
        self.RegisterData = None
        self.DateTimeData = None
        self.RawBatteryAndThermometerData = None
        self.CalBatteryAndThermometerData = None
        self.RawInertialAndMagneticData = None
        self.CalInertialAndMagneticData = None
        self.QuaternionData = None
        self.RotationMatrixData = None
        self.EulerAnglesData = None
        self.DigitalIOdata = None
        self.RawAnalogueInputData = None
        self.CalAnalogueInputData = None
        self.PWMoutputData = None
        self.RawADXL345busData = None
        self.CalADXL345busData = None

        # 모든 데이터 클래스를 동적으로 로드합니다.
        data_imported = False
        for class_name in ['ErrorDataClass', 'CommandDataClass', 'RegisterDataClass', 'DateTimeDataClass', 
                           'RawBatteryAndThermometerDataClass', 'CalBatteryAndThermometerDataClass', 
                           'RawInertialAndMagneticDataClass', 'CalInertialAndMagneticDataClass', 
                           'QuaternionDataClass', 'RotationMatrixDataClass', 'EulerAnglesDataClass', 
                           'DigitalIOdataClass', 'RawAnalogueInputDataClass', 'CalAnalogueInputDataClass', 
                           'PWMoutputDataClass', 'RawADXL345busDataClass', 'CalADXL345busDataClass']:
            try:
                class_ = globals()[class_name]
                setattr(self, class_name, class_(self.FileNamePrefix))
                data_imported = True
            except Exception as e:
                print(f"Failed to load {class_name}: {e}")

        if not data_imported:
            raise Exception('No data was imported.')

        # Register data에서 샘플 레이트를 가져옵니다.
        self.apply_sample_rate_from_register_data()

        # 명시적으로 SampleRate를 설정하는 추가 인수 처리
        self.process_additional_args(*args)

    def apply_sample_rate_from_register_data(self):
        try:
            # 예시: self.DateTimeData.SampleRate = self.sample_rate_from_reg_value(self.RegisterData.GetValueAtAddress(67))
            pass
        except Exception as e:
            print(e)

    def process_additional_args(self, *args):
        for i in range(0, len(args), 2):
            if args[i].endswith('SampleRate'):
                data_class_name = args[i].replace('SampleRate', 'Class')
                try:
                    data_class = getattr(self, data_class_name)
                    data_class.SampleRate = args[i + 1]
                except Exception as e:
                    print(f"Error setting sample rate for {data_class_name}: {e}")

    def plot(self):
        # 모든 데이터 클래스에 대해 plot 메서드를 호출합니다.
        for class_name in ['ErrorDataClass', 'CommandDataClass', 'RegisterDataClass', 'DateTimeDataClass', 
                           'RawBatteryAndThermometerDataClass', 'CalBatteryAndThermometerDataClass', 
                           'RawInertialAndMagneticDataClass', 'CalInertialAndMagneticDataClass', 
                           'QuaternionDataClass', 'RotationMatrixDataClass', 'EulerAnglesDataClass', 
                           'DigitalIOdataClass', 'RawAnalogueInputDataClass', 'CalAnalogueInputDataClass', 
                           'PWMoutputDataClass', 'RawADXL345busDataClass', 'CalADXL345busDataClass']:
            try:
                data_class = getattr(self, class_name)
                if data_class is not None:
                    data_class.plot()
            except Exception as e:
                print(f"Error plotting data for {class_name}: {e}")

    def sample_rate_from_reg_value(self, value):
        return 2**(value - 1)

        
'''

class xIMUdataClass:
    def __init__(self, *args):
        self.FileNamePrefix = ''
        self.ErrorData = []
        self.CommandData = []
        self.RegisterData = []
        self.DateTimeData = []
        self.RawBatteryAndThermometerData = []
        self.CalBatteryAndThermometerData = []
        self.RawInertialAndMagneticData = []
        self.CalInertialAndMagneticData = []
        self.QuaternionData = []
        self.RotationMatrixData = []
        self.EulerAnglesData = []
        self.DigitalIOdata = []
        self.RawAnalogueInputData = []
        self.CalAnalogueInputData = []
        self.PWMoutputData = []
        self.RawADXL345busData = []
        self.CalADXL345busData = []
        
        self.FileNamePrefix = args[0]
        dataImported = False
        try:
            self.ErrorData = ErrorDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.CommandData = CommandDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RegisterData = RegisterDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.DateTimeData = DateTimeDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RawBatteryAndThermometerData = RawBatteryAndThermometerDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.CalBatteryAndThermometerData = CalBatteryAndThermometerDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RawInertialAndMagneticData = RawInertialAndMagneticDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.CalInertialAndMagneticData = CalInertialAndMagneticDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.QuaternionData = QuaternionDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.EulerAnglesData = EulerAnglesDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RotationMatrixData = RotationMatrixDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.DigitalIOdata = DigitalIOdataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RawAnalogueInputData = RawAnalogueInputDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.CalAnalogueInputData = CalAnalogueInputDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.PWMoutputData = PWMoutputDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.RawADXL345busData = RawADXL345busDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        try:
            self.CalADXL345busData = CalADXL345busDataClass(self.FileNamePrefix)
            dataImported = True
        except:
            pass
        if not dataImported:
            raise Exception('No data was imported.')
        
        try:
            h = self.DateTimeData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(67))
        except:
            pass
        try:
            h = self.RawBatteryAndThermometerData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(68))
        except:
            pass
        try:
            h = self.CalBatteryAndThermometerData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(68))
        except:
            pass
        try:
            h = self.RawInertialAndMagneticData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(69))
        except:
            pass
        try:
            h = self.CalInertialAndMagneticData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(69))
        except:
            pass
        try:
            h = self.QuaternionData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(70))
        except:
            pass
        try:
            h = self.RotationMatrixData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(70))
        except:
            pass
        try:
            h = self.EulerAnglesData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(70))
        except:
            pass
        try:
            h = self.DigitalIOdata
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(78))
        except:
            pass
        try:
            h = self.RawAnalogueInputData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(80))
        except:
            pass
        try:
            h = self.CalAnalogueInputData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(80))
        except:
            pass
        try:
            h = self.RawADXL345busData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(85))
        except:
            pass
        try:
            h = self.CalADXL345busData
            h.SampleRate = self.SampleRateFromRegValue(self.RegisterData.GetValueAtAddress(85))
        except:
            pass
        
        for i in range(1, len(args), 2):
            if args[i] == 'DateTimeSampleRate':
                try:
                    h = self.DateTimeData
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'BattThermSampleRate':
                try:
                    h = self.RawBatteryAndThermometerData
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.CalBatteryAndThermometerData
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'InertialMagneticSampleRate':
                try:
                    h = self.RawInertialAndMagneticData
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.CalInertialAndMagneticData
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'QuaternionSampleRate':
                try:
                    h = self.QuaternionData
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.RotationMatrixData.SampleRate
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.EulerAnglesData
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'DigitalIOSampleRate':
                try:
                    h = self.DigitalIOdata
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'AnalogueInputSampleRate':
                try:
                    h = self.RawAnalogueInputData
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.CalAnalogueInputData
                    h.SampleRate = args[i+1]
                except:
                    pass
            elif args[i] == 'ADXL345SampleRate':
                try:
                    h = self.RawADXL345busData
                    h.SampleRate = args[i+1]
                except:
                    pass
                try:
                    h = self.CalADXL345busData
                    h.SampleRate = args[i+1]
                except:
                    pass
            else:
                raise Exception('Invalid argument.')
    
    def Plot(self):
        try:
            self.RawBatteryAndThermometerData.Plot()
        except:
            pass
        try:
            self.CalBatteryAndThermometerData.Plot()
        except:
            pass
        try:
            self.RawInertialAndMagneticData.Plot()
        except:
            pass
        try:
            self.CalInertialAndMagneticData.Plot()
        except:
            pass
        try:
            self.QuaternionData.Plot()
        except:
            pass
        try:
            self.EulerAnglesData.Plot()
        except:
            pass
        try:
            self.RotationMatrixDataClass.Plot()
        except:
            pass
        try:
            self.DigitalIOdata.Plot()
        except:
            pass
        try:
            self.RawAnalogueInputData.Plot()
        except:
            pass
        try:
            self.CalAnalogueInputData.Plot()
        except:
            pass
        try:
            self.RawADXL345busData.Plot()
        except:
            pass
        try:
            self.CalADXL345busData.Plot()
        except:
            pass
    
    def SampleRateFromRegValue(self, value):
        return int(2**(value-1))


In [49]:
class AHRS:
    def __init__(self, *args):
        self.SamplePeriod = 1/256
        self.Quaternion = [1, 0, 0, 0]
        self.Kp = 2
        self.Ki = 0
        self.KpInit = 200
        self.InitPeriod = 5
        self.q = [1, 0, 0, 0]
        self.IntError = [0, 0, 0]
        self.KpRamped = None
        
        for i in range(0, len(args), 2):
            if args[i] == 'SamplePeriod':
                self.SamplePeriod = args[i+1]
            elif args[i] == 'Quaternion':
                self.Quaternion = args[i+1]
                self.q = self.quaternConj(self.Quaternion)
            elif args[i] == 'Kp':
                self.Kp = args[i+1]
            elif args[i] == 'Ki':
                self.Ki = args[i+1]
            elif args[i] == 'KpInit':
                self.KpInit = args[i+1]
            elif args[i] == 'InitPeriod':
                self.InitPeriod = args[i+1]
            else:
                raise ValueError('Invalid argument')
        
        self.KpRamped = self.KpInit
    
    def Update(self, Gyroscope, Accelerometer, Magnetometer):
        raise NotImplementedError('This method is unimplemented')
    
    def UpdateIMU(self, Gyroscope, Accelerometer):
        if norm(Accelerometer) == 0:
            print('Accelerometer magnitude is zero. Algorithm update aborted.')
            return
        else:
            Accelerometer = Accelerometer / norm(Accelerometer)
        
        v = [2*(self.q[1]*self.q[4] - self.q[0]*self.q[3]),
             2*(self.q[0]*self.q[1] + self.q[2]*self.q[3]),
             self.q[0]**2 - self.q[1]**2 - self.q[2]**2 + self.q[3]**2]
        
        error = np.cross(v, Accelerometer)
        
        self.IntError = self.IntError + error
        
        Ref = Gyroscope - (self.Kp*error + self.Ki*self.IntError)
        
        pDot = 0.5 * self.quaternProd(self.q, [0, Ref[0], Ref[1], Ref[2]])
        
        self.q = self.q + pDot * self.SamplePeriod
        self.q = self.q / norm(self.q)
        
        self.Quaternion = self.quaternConj(self.q)
    
    def Reset(self):
        self.KpRamped = self.KpInit
        self.IntError = [0, 0, 0]
        self.q = [1, 0, 0, 0]
    
    def set_Quaternion(self, value):
        if norm(value) == 0:
            raise ValueError('Quaternion magnitude cannot be zero.')
        
        value = value / norm(value)
        self.Quaternion = value
        self.q = self.quaternConj(value)
    
    def quaternProd(self, a, b):
        ab = [a[0]*b[0] - a[1]*b[1] - a[2]*b[2] - a[3]*b[3],
              a[0]*b[1] + a[1]*b[0] + a[2]*b[3] - a[3]*b[2],
              a[0]*b[2] - a[1]*b[3] + a[2]*b[0] + a[3]*b[1],
              a[0]*b[3] + a[1]*b[2] - a[2]*b[1] + a[3]*b[0]]
        
        return ab
    
    def quaternConj(self, q):
        qConj = [q[0], -q[1], -q[2], -q[3]]
        return qConj




In [3]:
import numpy as np

class AHRS:
    def __init__(self, **kwargs):
        self.SamplePeriod = kwargs.get('SamplePeriod', 1/256)
        self.Quaternion = kwargs.get('Quaternion', np.array([1, 0, 0, 0]))
        self.Kp = kwargs.get('Kp', 2)
        self.Ki = kwargs.get('Ki', 0)
        self.KpInit = kwargs.get('KpInit', 200)
        self.InitPeriod = kwargs.get('InitPeriod', 5)
        self.q = np.array([1, 0, 0, 0])  # Internal quaternion
        self.IntError = np.array([0, 0, 0])  # Integral error
        self.KpRamped = self.KpInit

    def Update(self, Gyroscope, Accelerometer, Magnetometer):
        raise NotImplementedError("This method is unimplemented")

    def Update(self, Gyroscope, Accelerometer, Magnetometer):
        if np.linalg.norm(Accelerometer) == 0:
            print("Accelerometer magnitude is zero. Algorithm update aborted.")
            return
        else:
            Accelerometer = Accelerometer / np.linalg.norm(Accelerometer)

        v = np.array([2 * (self.q[1] * self.q[3] - self.q[0] * self.q[2]),
                    2 * (self.q[0] * self.q[1] + self.q[2] * self.q[3]),
                    self.q[0]**2 - self.q[1]**2 - self.q[2]**2 + self.q[3]**2])

        error = np.cross(v, Accelerometer)

        if self.KpRamped > self.Kp:
            self.IntError = np.array([0, 0, 0])
            self.KpRamped = self.KpRamped - (self.KpInit - self.Kp) / (self.InitPeriod / self.SamplePeriod)
        else:
            self.KpRamped = self.Kp
            self.IntError = self.IntError + error

        Ref = Gyroscope - (self.Kp * error + self.Ki * self.IntError)

        pDot = 0.5 * self.quaternProd(self.q, np.array([0, Ref[0], Ref[1], Ref[2]]))
        self.q = self.q + pDot * self.SamplePeriod
        self.q = self.q / np.linalg.norm(self.q)
        self.Quaternion = self.quaternConj(self.q)

    def Reset(self):
        self.KpRamped = self.KpInit
        self.IntError = np.array([0, 0, 0])
        self.q = np.array([1, 0, 0, 0])

    def quaternProd(self, a, b):
        ab = np.zeros(4)
        ab[0] = a[0] * b[0] - a[1] * b[1] - a[2] * b[2] - a[3] * b[3]
        ab[1] = a[0] * b[1] + a[1] * b[0] + a[2] * b[3] - a[3] * b[2]
        ab[2] = a[0] * b[2] - a[1] * b[3] + a[2] * b[0] + a[3] * b[1]
        ab[3] = a[0] * b[3] + a[1] * b[2] - a[2] * b[1] + a[3] * b[0]
        return ab

    def quaternConj(self, q):
        qConj = np.array([q[0], -q[1], -q[2], -q[3]])
        return qConj

In [52]:
#인공지능version
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

def SixDOFanimation(p, R, **kwargs):
    numSamples, _ = p.shape
    # 기본 매개변수 설정
    SamplePlotFreq = kwargs.get('SamplePlotFreq', 1)
    Trail = kwargs.get('Trail', 'Off')
    LimitRatio = kwargs.get('LimitRatio', 1)
    FullScreen = kwargs.get('FullScreen', False)
    View = kwargs.get('View', [30, 20])
    AxisLength = kwargs.get('AxisLength', 1)
    ShowArrowHead = kwargs.get('ShowArrowHead', True)
    Xlabel = kwargs.get('Xlabel', 'X')
    Ylabel = kwargs.get('Ylabel', 'Y')
    Zlabel = kwargs.get('Zlabel', 'Z')
    Title = kwargs.get('Title', '6DOF Animation')
    ShowLegend = kwargs.get('ShowLegend', True)
    CreateAVI = kwargs.get('CreateAVI', False)
    AVIfileName = kwargs.get('AVIfileName', '6DOF Animation')
    AVIfps = kwargs.get('AVIfps', 30)

    # 데이터 줄이기
    p = p[::SamplePlotFreq, :]
    R = R[:, :, ::SamplePlotFreq] * AxisLength
    numPlotSamples = p.shape[0]

    # 그래픽 설정
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel(Xlabel)
    ax.set_ylabel(Ylabel)
    ax.set_zlabel(Zlabel)
    ax.set_title(Title)

    # AVI 파일 생성을 위한 준비
    if CreateAVI:
        writer = FFMpegWriter(fps=AVIfps, metadata=dict(title=Title, artist='Matplotlib'), bitrate=1800)

    # 애니메이션 프레임 생성
    def update_graph(num):
        ax.view_init(View[0], View[1])
        ax.clear()

        # 축 한계 설정
        ax.set_xlim([np.min(p[:, 0])-AxisLength, np.max(p[:, 0])+AxisLength])
        ax.set_ylim([np.min(p[:, 1])-AxisLength, np.max(p[:, 1])+AxisLength])
        ax.set_zlim([np.min(p[:, 2])-AxisLength, np.max(p[:, 2])+AxisLength])

        # 트레일 그리기
        if Trail == 'DotsOnly' or Trail == 'All':
            ax.scatter(p[:num, 0], p[:num, 1], p[:num, 2], c='k')

        # 화살표(Quiver) 플롯 추가
        if num > 0:
            ax.quiver(p[num-1, 0], p[num-1, 1], p[num-1, 2], R[0, :, num-1], R[1, :, num-1], R[2, :, num-1], color=['r', 'g', 'b'], length=AxisLength)

        # 레전드 표시
        if ShowLegend:
            ax.legend(['X-axis', 'Y-axis', 'Z-axis'])

    # 애니메이션 생성
    ani = matplotlib.animation.FuncAnimation(fig, update_graph, numPlotSamples, interval=1000/AVIfps, blit=False)

    # AVI 파일 저장
    if CreateAVI:
        ani.save(AVIfileName + '.mp4', writer=writer)

    plt.show()

In [ ]:
#원본
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def SixDOFanimation(p, R, **kwargs):
    ## Create local variables
    # Required arguments
    p = args[0]                # position of body
    R = args[1]                # rotation matrix of body
    numSamples = p.shape[0]
    
    # 기본값 설정
    SamplePlotFreq = kwargs.get('SamplePlotFreq', 1)
    Trail = kwargs.get('Trail', 'Off')
    LimitRatio = kwargs.get('LimitRatio', 1)
    Position = kwargs.get('Position', [])
    FullScreen = kwargs.get('FullScreen', False)
    View = kwargs.get('View', [30, 20])
    AxisLength = kwargs.get('AxisLength', 1)
    ShowArrowHead = kwargs.get('ShowArrowHead', 'on')
    Xlabel = kwargs.get('Xlabel', 'X')
    Ylabel = kwargs.get('Ylabel', 'Y')
    Zlabel = kwargs.get('Zlabel', 'Z')
    Title = kwargs.get('Title', '6DOF Animation')
    ShowLegend = kwargs.get('ShowLegend', True)

    CreateAVI = False
    AVIfileName = '6DOF Animation'
    AVIfileNameEnum = True
    AVIfps = 30
    for i in range(2, len(args), 2):
        if args[i] == 'SamplePlotFreq':
            SamplePlotFreq = args[i+1]
        elif args[i] == 'Trail':
            Trail = args[i+1]
            if Trail != 'Off' and Trail != 'DotsOnly' and Trail != 'All':
                raise ValueError('Invalid argument. Trail must be \'Off\', \'DotsOnly\' or \'All\'.')
        elif args[i] == 'LimitRatio':
            LimitRatio = args[i+1]
        elif args[i] == 'Position':
            Position = args[i+1]
        elif args[i] == 'FullScreen':
            FullScreen = args[i+1]
        elif args[i] == 'View':
            View = args[i+1]
        elif args[i] == 'AxisLength':
            AxisLength = args[i+1]
        elif args[i] == 'ShowArrowHead':
            ShowArrowHead = args[i+1]
        elif args[i] == 'Xlabel':
            Xlabel = args[i+1]
        elif args[i] == 'Ylabel':
            Ylabel = args[i+1]
        elif args[i] == 'Zlabel':
            Zlabel = args[i+1]
        elif args[i] == 'Title':
            Title = args[i+1]
        elif args[i] == 'ShowLegend':
            ShowLegend = args[i+1]
        elif args[i] == 'CreateAVI':
            CreateAVI = args[i+1]
        elif args[i] == 'AVIfileName':
            AVIfileName = args[i+1]
        elif args[i] == 'AVIfileNameEnum':
            AVIfileNameEnum = args[i+1]
        elif args[i] == 'AVIfps':
            AVIfps = args[i+1]
        else:
            raise ValueError('Invalid argument.')
    # 데이터를 샘플링 주파수에 따라 줄입니다.
    p = p[::SamplePlotFreq, :]
    R = R[:, :, ::SamplePlotFreq] * AxisLength
    numPlotSamples = p.shape[0]
    if View.size > 2:
        View = View[::SamplePlotFreq, :]
    numPlotSamples, _ = p.shape
    ## Setup AVI file
    aviobj = None                                                            	# create null object
    if CreateAVI:
        fileName = AVIfileName + '.avi'
        if os.path.exists(fileName):
            if AVIfileNameEnum:                                              	# if file name exists and enum enabled
                i = 0
                while os.path.exists(fileName):                                  # find un-used file name by appending enum
                    fileName = AVIfileName + str(i) + '.avi'
                    i = i + 1
            else:                                                                # else file name exists and enum disabled
                fileName = None                                                  # file will not be created
        if fileName is None:
            print('AVI file not created as file already exists.')
        else:
            aviobj = avifile(fileName, 'fps', AVIfps, 'compression', 'Cinepak', 'quality', 100)
    ## Setup figure and plot
    # Create figure
    fig = figure('NumberTitle', 'off', 'Name', '6DOF Animation')
    if FullScreen:
        screenSize = get(0, 'ScreenSize')
        set(fig, 'Position', [0, 0, screenSize[3], screenSize[4]])
    elif Position:
        set(fig, 'Position', Position)
    set(gca, 'drawmode', 'fast')
    lighting phong
    set(gcf, 'Renderer', 'zbuffer')
    hold on
    axis equal
    grid on
    view(View[0, 0], View[0, 1])
    title(i)
    xlabel(Xlabel)
    ylabel(Ylabel)
    zlabel(Zlabel)
    # Create plot data arrays
    if Trail == 'DotsOnly' or Trail == 'All':
        x = zeros(numPlotSamples, 1)
        y = zeros(numPlotSamples, 1)
        z = zeros(numPlotSamples, 1)
    if Trail == 'All':
        ox = zeros(numPlotSamples, 1)
        oy = zeros(numPlotSamples, 1)
        oz = zeros(numPlotSamples, 1)
        ux = zeros(numPlotSamples, 1)
        vx = zeros(numPlotSamples, 1)
        wx = zeros(numPlotSamples, 1)
        uy = zeros(numPlotSamples, 1)
        vy = zeros(numPlotSamples, 1)
        wy = zeros(numPlotSamples, 1)
        uz = zeros(numPlotSamples, 1)
        vz = zeros(numPlotSamples, 1)
        wz = zeros(numPlotSamples, 1)
    x[0] = p[0, 0]
    y[0] = p[0, 1]
    z[0] = p[0, 2]
    ox[0] = x[0]
    oy[0] = y[0]
    oz[0] = z[0]
    ux[0] = R[0, 0, 0:1]
    vx[0] = R[1, 0, 0:1]
    wx[0] = R[2, 0, 0:1]
    uy[0] = R[0, 1, 0:1]
    vy[0] = R[1, 1, 0:1]
    wy[0] = R[2, 1, 0:1]
    uz[0] = R[0, 2, 0:1]
    vz[0] = R[1, 2, 0:1]
    wz[0] = R[2, 2, 0:1]
    # Create graphics handles
    orgHandle = plot3(x, y, z, 'k.')
    if ShowArrowHead:
        ShowArrowHeadStr = 'on'
    else:
        ShowArrowHeadStr = 'off'
    quivXhandle = quiver3(ox, oy, oz, ux, vx, wx,  'r', 'ShowArrowHead', ShowArrowHeadStr, 'MaxHeadSize', 0.999999, 'AutoScale', 'off')
    quivYhandle = quiver3(ox, oy, oz, uy, vy, wy,  'g', 'ShowArrowHead', ShowArrowHeadStr, 'MaxHeadSize', 0.999999, 'AutoScale', 'off')
    quivZhandle = quiver3(ox, ox, oz, uz, vz, wz,  'b', 'ShowArrowHead', ShowArrowHeadStr, 'MaxHeadSize', 0.999999, 'AutoScale', 'off')
    # Create legend
    if ShowLegend:
        legend('Origin', 'X', 'Y', 'Z')
    # Set initial limits
    Xlim = [x[0]-AxisLength, x[0]+AxisLength] * LimitRatio
    Ylim = [y[0]-AxisLength, y[0]+AxisLength] * LimitRatio
    Zlim = [z[0]-AxisLength, z[0]+AxisLength] * LimitRatio
    set(gca, 'Xlim', Xlim, 'Ylim', Ylim, 'Zlim', Zlim)
    # Set initial view
    view(View[0, :])
    ## Plot one sample at a time
    for i in range(numPlotSamples):
        # Update graph title
        if Title == '':
            titleText = 'Sample {} of {}'.format(1+((i-1)*SamplePlotFreq), numSamples)
        else:
            titleText = '{} ({})'.format(Title, 'Sample {} of {}'.format(1+((i-1)*SamplePlotFreq), numSamples))
        title(titleText)
        # Plot body x y z axes
        if Trail == 'DotsOnly' or Trail == 'All':
            x[0:i] = p[0:i, 0]
            y[0:i] = p[0:i, 1]
            z[0:i] = p[0:i, 2]
        else:
            x = p[i, 0]
            y = p[i, 1]
            z = p[i, 2]
        if Trail == 'All':
            ox[0:i] = p[0:i, 0]
            oy[0:i] = p[0:i, 1]
            oz[0:i] = p[0:i, 2]
            ux[0:i] = R[0, 0, 0:i]
            vx[0:i] = R[1, 0, 0:i]
            wx[0:i] = R[2, 0, 0:i]
            uy[0:i] = R[0, 1, 0:i]
            vy[0:i] = R[1, 1, 0:i]
            wy[0:i] = R[2, 1, 0:i]
            uz[0:i] = R[0, 2, 0:i]
            vz[0:i] = R[1, 2, 0:i]
            wz[0:i] = R[2, 2, 0:i]
        else:
            ox = p[i, 0]
            oy = p[i, 1]
            oz = p[i, 2]
            ux = R[0, 0, i]
            vx = R[1, 0, i]
            wx = R[2, 0, i]
            uy = R[0, 1, i]
            vy = R[1, 1, i]
            wy = R[2, 1, i]
            uz = R[0, 2, i]
            vz = R[1, 2, i]
            wz = R[2, 2, i]
        set(orgHandle, 'xdata', x, 'ydata', y, 'zdata', z)
        set(quivXhandle, 'xdata', ox, 'ydata', oy, 'zdata', oz,'udata', ux, 'vdata', vx, 'wdata', wx)
        set(quivYhandle, 'xdata', ox, 'ydata', oy, 'zdata', oz,'udata', uy, 'vdata', vy, 'wdata', wy)
        set(quivZhandle, 'xdata', ox, 'ydata', oy, 'zdata', oz,'udata', uz, 'vdata', vz, 'wdata', wz)
        # Adjust axes for snug fit and draw
        axisLimChanged = False
        if p[i, 0] - AxisLength < Xlim[0]:
            Xlim[0] = p[i, 0] - LimitRatio*AxisLength
            axisLimChanged = True
        if p[i, 1] - AxisLength < Ylim[0]:
            Ylim[0] = p[i, 1] - LimitRatio*AxisLength
            axisLimChanged = True
        if p[i, 2] - AxisLength < Zlim[0]:
            Zlim[0] = p[i, 2] - LimitRatio*AxisLength
            axisLimChanged = True
        if p[i, 0] + AxisLength > Xlim[1]:
            Xlim[1] = p[i, 0] + LimitRatio*AxisLength
            axisLimChanged = True
        if p[i, 1] + AxisLength > Ylim[1]:
            Ylim[1] = p[i, 1] + LimitRatio*AxisLength
            axisLimChanged = True
        if p[i, 2] + AxisLength > Zlim[1]:
            Zlim[1] = p[i, 2] + LimitRatio*AxisLength
            axisLimChanged = True
        if axisLimChanged:
            set(gca, 'Xlim', Xlim, 'Ylim', Ylim, 'Zlim', Zlim)
        drawnow()
        # Adjust view
        if View.size > 2:
            view(View[i, :])
        # Add frame to AVI object
        if aviobj is not None:
            frame = getframe(fig)
            aviobj = addframe(aviobj, frame)
    hold off
    # Close AVI file
    if aviobj is not None:
        aviobj = close(aviobj)
    return fig




In [54]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class SixDOFanimation:
    def __init__(self, p, R, **kwargs):
        # Initialize parameters and variables
        self.p = p  # Position of body
        self.R = R  # Rotation matrix of body
        self.numSamples = p.shape[0]
        
        # Default values of optional arguments
        self.samplePlotFreq = kwargs.get('SamplePlotFreq', 1)
        self.trail = kwargs.get('Trail', 'Off')
        self.limitRatio = kwargs.get('LimitRatio', 1)
        self.position = kwargs.get('Position', None)
        self.fullScreen = kwargs.get('FullScreen', False)
        self.view = kwargs.get('View', [30, 20])
        self.axisLength = kwargs.get('AxisLength', 1)
        self.showArrowHead = kwargs.get('ShowArrowHead', True)
        self.xlabel = kwargs.get('Xlabel', 'X')
        self.ylabel = kwargs.get('Ylabel', 'Y')
        self.zlabel = kwargs.get('Zlabel', 'Z')
        self.title = kwargs.get('Title', '6DOF Animation')
        self.showLegend = kwargs.get('ShowLegend', True)
        
        # Initialize figure
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlabel(self.xlabel)
        self.ax.set_ylabel(self.ylabel)
        self.ax.set_zlabel(self.zlabel)
        self.ax.set_title(self.title)
        if self.showLegend:
            self.ax.legend(['Origin', 'X', 'Y', 'Z'])

        # Set view
        self.ax.view_init(elev=self.view[0], azim=self.view[1])

        # Initialize quiver for arrows
        self.quiver = None
        
    def update_plot(self, i):
        # Update the plot for each frame
        p = self.p[i]
        R = self.R[:, :, i] * self.axisLength
        
        if self.quiver is None:
            # Initialize quiver
            self.quiver = [self.ax.quiver(p[0], p[1], p[2], R[0, j], R[1, j], R[2, j], color=color) for j, color in enumerate(['r', 'g', 'b'])]
        else:
            # Update quiver
            for j in range(3):
                self.quiver[j].remove()
                self.quiver[j] = self.ax.quiver(p[0], p[1], p[2], R[0, j], R[1, j], R[2, j], color=['r', 'g', 'b'][j])
        
        # Set the limits
        self.ax.set_xlim([p[0] - self.axisLength, p[0] + self.axisLength])
        self.ax.set_ylim([p[1] - self.axisLength, p[1] + self.axisLength])
        self.ax.set_zlim([p[2] - self.axisLength, p[2] + self.axisLength])

    def animate(self):
        # Create the animation
        anim = FuncAnimation(self.fig, self.update_plot, frames=self.numSamples, interval=1000/self.samplePlotFreq)
        plt.show()


In [45]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from ahrs.filters import Madgwick
from ahrs.common.orientation import q2R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 데이터 불러오기
file_path = r'D:\20192230 이동섭\IMU sensor\Gait-Tracking-With-x-IMU-master\Gait Tracking With x-IMU\Datasets\stairsAndCorridor_CalInertialAndMag.csv'
data = pd.read_csv(file_path)

# 센서 데이터 추출
acc_data = data[['Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']].values
gyro_data = data[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)']].values * np.pi / 180  # 라디안으로 변환
mag_data = data[['Magnetometer X (G)', 'Magnetometer Y (G)', 'Magnetometer Z (G)']].values

# Madgwick 알고리즘 초기화
sample_rate = 256  # Hz
madgwick = Madgwick(frequency=sample_rate, beta=None)

# 자세 계산 (쿼터니언)
quaternions = np.array([madgwick.update_imu(gyro=gyro, accel=acc) for gyro, acc in zip(gyro_data, acc_data)])


# 가속도 데이터를 지구 좌표계로 변환
earth_acc = np.array([q2R(q).dot(acc) for q, acc in zip(quaternions, acc_data)])

# 가속도 적분
velocity = np.cumsum(earth_acc, axis=0) / sample_rate
position = np.cumsum(velocity, axis=0) / sample_rate

# 3D 트랙 플로팅
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(position[:, 0], position[:, 1], position[:, 2])
plt.show()

AttributeError: 'Madgwick' object has no attribute 'update_imu'

In [55]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from ahrs.filters import AHRS, Madgwick
from ahrs.common.orientation import q2R
from mpl_toolkits.mplot3d import Axes3D
import quatern2rotMat, quaternConj, quaternRotate

# 필요한 함수들을 Python으로 변환합니다.
def butter_highpass(cutoff, fs, order=1):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_lowpass(cutoff, fs, order=1):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# AHRS 알고리즘 클래스를 Python으로 변환합니다.
class AHRS:
    # 클래스 초기화와 필요한 메서드들을 여기에 구현하세요.
    pass

# 데이터를 불러오는 부분입니다.
# 실제 데이터 경로와 변수들을 설정합니다.
filePath = 'Datasets/stairsAndCorridor'
startTime = 6
stopTime = 26

# 이 부분에서 데이터를 불러오고 처리합니다.
# 데이터 불러오기에 필요한 함수들을 Python으로 변환하거나 적절한 라이브러리를 사용합니다.

# 시뮬레이션 데이터를 생성합니다.
time = np.linspace(0, 10, 1000)  # 예시 시간 벡터
gyrX, gyrY, gyrZ = np.random.rand(3, 1000)  # 예시 자이로스코프 데이터
accX, accY, accZ = np.random.rand(3, 1000)  # 예시 가속도계 데이터

# 가속도 데이터의 크기를 계산하고 필터링합니다.
acc_mag = np.sqrt(accX**2 + accY**2 + accZ**2)
b, a = butter_highpass(0.001, 1/256)
acc_magFilt = filtfilt(b, a, acc_mag)
acc_magFilt = np.abs(acc_magFilt)
b, a = butter_lowpass(5, 256)
acc_magFilt = filtfilt(b, a, acc_magFilt)
stationary = acc_magFilt < 0.05

# 자세 추정을 위한 AHRS 알고리즘을 초기화하고 사용합니다.
# AHRS 알고리즘을 구현하거나 적절한 라이브러리를 사용하세요.

# 이동을 추정하기 위해 가속도 데이터를 처리합니다.

# 속도와 위치를 추정합니다.

# 결과를 플로팅합니다.
plt.figure()
plt.subplot(211)
plt.title('Gyroscope')
plt.plot(time, gyrX, label='X')
plt.plot(time, gyrY, label='Y')
plt.plot(time, gyrZ, label='Z')
plt.legend()

plt.subplot(212)
plt.title('Accelerometer')
plt.plot(time, accX, label='X')
plt.plot(time, accY, label='Y')
plt.plot(time, accZ, label='Z')
plt.plot(time, acc_magFilt, label='Filtered')
plt.plot(time, stationary, label='Stationary')
plt.legend()

plt.show()

ImportError: cannot import name 'AHRS' from 'ahrs.filters' (c:\Users\Lab419_6\anaconda3\lib\site-packages\ahrs\filters\__init__.py)

In [59]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Select dataset (comment in/out)
filePath = r'D:\20192230 이동섭\IMU sensor\Gait-Tracking-With-x-IMU-master\Gait Tracking With x-IMU\Datasets\stairsAndCorridor_CalInertialAndMag.csv'
startTime = 6
stopTime = 26
# filePath = 'Datasets/stairsAndCorridor'
# startTime = 5
# stopTime = 53
# filePath = 'Datasets/spiralStairs'
# startTime = 4
# stopTime = 47

# Import data
samplePeriod = 1/256
xIMUdata = xIMUdataClass(filePath, 'InertialMagneticSampleRate', 1/samplePeriod)
time = xIMUdata.CalInertialAndMagneticData.Time
gyrX = xIMUdata.CalInertialAndMagneticData.Gyroscope.X
gyrY = xIMUdata.CalInertialAndMagneticData.Gyroscope.Y
gyrZ = xIMUdata.CalInertialAndMagneticData.Gyroscope.Z
accX = xIMUdata.CalInertialAndMagneticData.Accelerometer.X
accY = xIMUdata.CalInertialAndMagneticData.Accelerometer.Y
accZ = xIMUdata.CalInertialAndMagneticData.Accelerometer.Z
del xIMUdata

# Manually frame data
# startTime = 0
# stopTime = 10
indexSel = np.where((startTime <= time) & (time <= stopTime))[0]
time = time[indexSel]
gyrX = gyrX[indexSel]
gyrY = gyrY[indexSel]
gyrZ = gyrZ[indexSel]
accX = accX[indexSel]
accY = accY[indexSel]
accZ = accZ[indexSel]

# Detect stationary periods
# Compute accelerometer magnitude
acc_mag = np.sqrt(accX*accX + accY*accY + accZ*accZ)
# HP filter accelerometer data
filtCutOff = 0.001
b, a = butter(1, (2*filtCutOff)/(1/samplePeriod), 'high')
acc_magFilt = filtfilt(b, a, acc_mag)
# Compute absolute value
acc_magFilt = np.abs(acc_magFilt)
# LP filter accelerometer data
filtCutOff = 5
b, a = butter(1, (2*filtCutOff)/(1/samplePeriod), 'low')
acc_magFilt = filtfilt(b, a, acc_magFilt)
# Threshold detection
stationary = acc_magFilt < 0.05

# Plot data raw sensor data and stationary periods
fig, ax = plt.subplots(2, 1, figsize=(9, 6))
ax[0].plot(time, gyrX, 'r')
ax[0].plot(time, gyrY, 'g')
ax[0].plot(time, gyrZ, 'b')
ax[0].set_title('Gyroscope')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Angular velocity (°/s)')
ax[0].legend(['X', 'Y', 'Z'])
ax[1].plot(time, accX, 'r')
ax[1].plot(time, accY, 'g')
ax[1].plot(time, accZ, 'b')
ax[1].plot(time, acc_magFilt, ':k')
ax[1].plot(time, stationary, 'k', linewidth=2)
ax[1].set_title('Accelerometer')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Acceleration (g)')
ax[1].legend(['X', 'Y', 'Z', 'Filtered', 'Stationary'])
plt.tight_layout()
plt.show()

# Compute orientation
quat = np.zeros((len(time), 4))
AHRSalgorithm = AHRS('SamplePeriod', 1/256, 'Kp', 1, 'KpInit', 1)
# Initial convergence
initPeriod = 2
indexSel = np.arange(0, np.where(np.sign(time-(time[0]+initPeriod))+1)[0][0])
for i in range(2000):
    AHRSalgorithm.UpdateIMU([0, 0, 0], [np.mean(accX[indexSel]), np.mean(accY[indexSel]), np.mean(accZ[indexSel])])
# For all data
for t in range(len(time)):
    if(stationary[t]):
        AHRSalgorithm.Kp = 0.5
    else:
        AHRSalgorithm.Kp = 0
    AHRSalgorithm.UpdateIMU(np.deg2rad([gyrX[t], gyrY[t], gyrZ[t]]), [accX[t], accY[t], accZ[t]])
    quat[t,:] = AHRSalgorithm.Quaternion

# Compute translational accelerations
# Rotate body accelerations to Earth frame
acc = quaternRotate(np.column_stack((accX, accY, accZ)), quaternConj(quat))
# Convert acceleration measurements to m/s/s
acc = acc * 9.81

# Plot translational accelerations
plt.figure(figsize=(9, 3))
plt.plot(time, acc[:,0], 'r')
plt.plot(time, acc[:,1], 'g')
plt.plot(time, acc[:,2], 'b')
plt.title('Acceleration')
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (m/s/s)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# Compute translational velocities
acc[:,2] = acc[:,2] - 9.81
# Integrate acceleration to yield velocity
vel = np.zeros_like(acc)
for t in range(1, len(vel)):
    vel[t,:] = vel[t-1,:] + acc[t,:] * samplePeriod
    if(stationary[t] == 1):
        vel[t,:] = [0, 0, 0]     # force zero velocity when foot stationary

# Compute integral drift during non-stationary periods
velDrift = np.zeros_like(vel)
stationaryStart = np.where(np.diff(stationary) == -1)[0]
stationaryEnd = np.where(np.diff(stationary) == 1)[0]
for i in range(len(stationaryEnd)):
    driftRate = vel[stationaryEnd[i]-1, :] / (stationaryEnd[i] - stationaryStart[i])
    enum = np.arange(1, stationaryEnd[i] - stationaryStart[i] + 1)
    drift = np.column_stack((enum*driftRate[0], enum*driftRate[1], enum*driftRate[2]))
    velDrift[stationaryStart[i]:stationaryEnd[i]-1, :] = drift

# Remove integral drift
vel = vel - velDrift

# Plot translational velocity
plt.figure(figsize=(9, 3))
plt.plot(time, vel[:,0], 'r')
plt.plot(time, vel[:,1], 'g')
plt.plot(time, vel[:,2], 'b')
plt.title('Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# Compute translational position
# Integrate velocity to yield position
pos = np.zeros_like(vel)
for t in range(1, len(pos)):
    pos[t,:] = pos[t-1,:] + vel[t,:] * samplePeriod

# Plot translational position
plt.figure(figsize=(9, 6))
plt.plot(time, pos[:,0], 'r')
plt.plot(time, pos[:,1], 'g')
plt.plot(time, pos[:,2], 'b')
plt.title('Position')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# Plot 3D foot trajectory
# posPlot = pos[np.where(~stationary)[0], :]
# quatPlot = quat[np.where(~stationary)[0], :]
posPlot = pos
quatPlot = quat
# Extend final sample to delay end of animation
extraTime = 20
onesVector = np.ones(int(extraTime*(1/samplePeriod)))
posPlot = np.vstack((posPlot, [posPlot[-1, 0]*onesVector, posPlot[-1, 1]*onesVector, posPlot[-1, 2]*onesVector]))
quatPlot = np.vstack((quatPlot, [quatPlot[-1, 0]*onesVector, quatPlot[-1, 1]*onesVector, quatPlot[-1, 2]*onesVector, quatPlot[-1, 3]*onesVector]))
# Create 6 DOF animation
SamplePlotFreq = 4
Spin = 120

'''
SixDOFanimation(posPlot, quatern2rotMat(quatPlot), \
                'SamplePlotFreq', SamplePlotFreq, 'Trail', 'All', \
                'Position', [9 39 1280 768], 'View', [(100:(Spin/(length(posPlot)-1)):(100+Spin))', 10*ones(length(posPlot), 1)], \
                'AxisLength', 0.1, 'ShowArrowHead', false, \
                'Xlabel', 'X (m)', 'Ylabel', 'Y (m)', 'Zlabel', 'Z (m)', 'ShowLegend', false, \
                'CreateAVI', false, 'AVIfileNameEnum', false, 'AVIfps', ((1/samplePeriod) / SamplePlotFreq))
'''



Exception: No data was imported.

In [56]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# 데이터셋 선택
filePath = r'D:\20192230 이동섭\IMU sensor\Gait-Tracking-With-x-IMU-master\Gait Tracking With x-IMU\Datasets\stairsAndCorridor_CalInertialAndMag.csv'
startTime = 6
stopTime = 26

# 데이터 가져오기 (xIMUdataClass 함수 구현 필요)
samplePeriod = 1/256
xIMUdata = xIMUdataClass(filePath, 'InertialMagneticSampleRate', 1/samplePeriod)
time = xIMUdata.CalInertialAndMagneticData.Time
gyrX = xIMUdata.CalInertialAndMagneticData.Gyroscope.X
gyrY = xIMUdata.CalInertialAndMagneticData.Gyroscope.Y
gyrZ = xIMUdata.CalInertialAndMagneticData.Gyroscope.Z
accX = xIMUdata.CalInertialAndMagneticData.Accelerometer.X
accY = xIMUdata.CalInertialAndMagneticData.Accelerometer.Y
accZ = xIMUdata.CalInertialAndMagneticData.Accelerometer.Z
del xIMUdata

# 데이터 수동 프레이밍
startIndex = np.where(time >= startTime)[0][0]
stopIndex = np.where(time <= stopTime)[0][-1]
time = time[startIndex:stopIndex+1]
gyrX = gyrX[startIndex:stopIndex+1]
gyrY = gyrY[startIndex:stopIndex+1]
gyrZ = gyrZ[startIndex:stopIndex+1]
accX = accX[startIndex:stopIndex+1]
accY = accY[startIndex:stopIndex+1]
accZ = accZ[startIndex:stopIndex+1]

# 정지 구간 감지
acc_mag = np.sqrt(accX**2 + accY**2 + accZ**2)
filtCutOff = 0.001
b, a = butter(1, (2*filtCutOff)/(1/samplePeriod), 'high')
acc_magFilt = filtfilt(b, a, acc_mag)
acc_magFilt = np.abs(acc_magFilt)
filtCutOff = 5
b, a = butter(1, (2*filtCutOff)/(1/samplePeriod), 'low')
acc_magFilt = filtfilt(b, a, acc_magFilt)
stationary = acc_magFilt < 0.05

# 시각화: 원시 센서 데이터 및 정지 구간
plt.figure(figsize=(15, 10))
plt.subplot(2, 1, 1)
plt.title('Gyroscope')
plt.plot(time, gyrX, 'r')
plt.plot(time, gyrY, 'g')
plt.plot(time, gyrZ, 'b')
plt.xlabel('Time (s)')
plt.ylabel('Angular velocity (°/s)')
plt.legend(['X', 'Y', 'Z'])

plt.subplot(2, 1, 2)
plt.title('Accelerometer')
plt.plot(time, accX, 'r')
plt.plot(time, accY, 'g')
plt.plot(time, accZ, 'b')
plt.plot(time, acc_magFilt, ':k')
plt.plot(time, stationary, 'k', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (g)')
plt.legend(['X', 'Y', 'Z', 'Filtered', 'Stationary'])
plt.tight_layout()
plt.show()

# 자세 계산 (AHRS 알고리즘 구현 필요)
quat = np.zeros((len(time), 4))
AHRSalgorithm = AHRS('SamplePeriod', 1/256, 'Kp', 1, 'KpInit', 1)
initPeriod = 2
indexSel = np.arange(0, np.where(time >= (time[0]+initPeriod))[0][0])
for i in range(2000):
    AHRSalgorithm.UpdateIMU([0, 0, 0], [np.mean(accX[indexSel]), np.mean(accY[indexSel]), np.mean(accZ[indexSel])])

for t in range(len(time)):
    if stationary[t]:
        AHRSalgorithm.Kp = 0.5
    else:
        AHRSalgorithm.Kp = 0
    AHRSalgorithm.UpdateIMU(np.deg2rad([gyrX[t], gyrY[t], gyrZ[t]]), [accX[t], accY[t], accZ[t]])
    quat[t, :] = AHRSalgorithm.Quaternion

# 가속도 변환 계산 (quaternRotate 및 quaternConj 함수 구현 필요)
acc = quaternRotate(np.column_stack((accX, accY, accZ)), quaternConj(quat))
acc *= 9.81

# 시각화: 이동 가속도
plt.figure(figsize=(9, 3))
plt.plot(time, acc[:,0], 'r')
plt.plot(time, acc[:,1], 'g')
plt.plot(time, acc[:,2], 'b')
plt.title('Acceleration')
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (m/s²)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# 이동 속도 계산
acc[:,2] -= 9.81
vel = np.zeros_like(acc)
for t in range(1, len(vel)):
    vel[t, :] = vel[t-1, :] + acc[t, :] * samplePeriod
    if stationary[t]:
        vel[t, :] = 0  # 정지 구간에서 속도를 0으로 설정

# 시각화: 이동 속도
plt.figure(figsize=(9, 3))
plt.plot(time, vel[:,0], 'r')
plt.plot(time, vel[:,1], 'g')
plt.plot(time, vel[:,2], 'b')
plt.title('Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# 이동 위치 계산
pos = np.zeros_like(vel)
for t in range(1, len(pos)):
    pos[t, :] = pos[t-1, :] + vel[t, :] * samplePeriod

# 시각화: 이동 위치
plt.figure(figsize=(9, 6))
plt.plot(time, pos[:,0], 'r')
plt.plot(time, pos[:,1], 'g')
plt.plot(time, pos[:,2], 'b')
plt.title('Position')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend(['X', 'Y', 'Z'])
plt.show()

# 3D 발자국 궤적 애니메이션 생성 (SixDOFanimation 함수 구현 필요)
posPlot = pos
quatPlot = quat
extraTime = 20
onesVector = np.ones(int(extraTime * (1/samplePeriod)))
posPlot = np.vstack((posPlot, np.outer(onesVector, posPlot[-1])))
quatPlot = np.vstack((quatPlot, np.outer(onesVector, quatPlot[-1])))

# SixDOFanimation 함수 사용 - 해당 함수는 정의되어 있어야 합니다.
SixDOFanimation(posPlot, quatern2rotMat(quatPlot), SamplePlotFreq=4, Trail='All', Position=[9, 39, 1280, 768], View=[100, 10], AxisLength=0.1, ShowArrowHead=False, Xlabel='X (m)', Ylabel='Y (m)', Zlabel='Z (m)', ShowLegend=False, CreateAVI=False, AVIfileNameEnum=False, AVIfps=(1/samplePeriod) / 4)

Exception: No data was imported.